<a href="https://colab.research.google.com/github/ochouchene/-Data-Gathering-with-API-/blob/main/Support_vector_machines_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = "https://i.imgur.com/IpuCW3s.jpg"  # Make sure to download CSV if needed
df = pd.read_csv("Electric_cars_dataset.csv")
df.head()

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4315 entries, 0 to 4314
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 4315 non-null   object 
 1   VIN (1-10)                                         4315 non-null   object 
 2   County                                             4314 non-null   object 
 3   City                                               4315 non-null   object 
 4   State                                              4313 non-null   object 
 5   ZIP Code                                           4315 non-null   float64
 6   Model Year                                         4315 non-null   float64
 7   Make                                               4315 non-null   object 
 8   Model                                              4313 non-null   object 
 9   Electric

In [ ]:
df.describe()

,ZIP Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID
count,4315.000000,4315.000000,4315.000000,4315.000000,4305.000000,4.315000e+03
mean,98192.125145,2018.173581,106.699189,2276.170336,30.054123,1.967934e+08
std,2210.586917,2.752459,103.538532,11213.895536,14.571708,1.092120e+08
min,9751.000000,1993.000000,0.000000,0.000000,1.000000,4.777000e+03
25%,98052.000000,2016.000000,14.000000,0.000000,18.000000,1.340729e+08
50%,98122.000000,2018.000000,73.000000,0.000000,34.000000,1.725798e+08
75%,98370.000000,2021.000000,215.000000,0.000000,43.000000,2.297611e+08
max,99567.000000,2022.000000,337.000000,110950.000000,49.000000,4.789346e+08


In [ ]:
df.isnull().sum()

,0
ID,0
VIN (1-10),0
County,1
City,0
State,2
ZIP Code,0
Model Year,0
Make,0
Model,2
Electric Vehicle Type,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
profile = pandas_profiling.ProfileReport(df, title="Electric Vehicles Data Profiling", explorative=True)
profile.to_file("EV_data_report.html")

NameError: name 'pandas_profiling' is not defined

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = df.select_dtypes(include=['object']).columns


for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
target = 'Expected Price ($1k)'
features = df.drop(columns=[target])
X = features
y = df[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_model.fit(X_train_scaled, y_train)

SVR(C=100, gamma=0.1)

In [ ]:
y_pred = svr_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

RMSE: 22.733848161058383
R2 Score: 0.7258607024943375
